# Prohledávání hyperparametrů pro model BiLSTM nad datasetem SST2 

Tento notebook slouží k nalezení optimálních hyperparametrů nad datasetem SST2 pro model BiLSTM s odemčenou embedding vrstvou. Hyperparametry jsou hledány pro původní i augmentovaný dataset pro normální trénink i destilaci.

K prohledávání je využito knihovny Optuna s algoritmem Hyperband. Nejlepší konfigurace je volena na základě F1-skóre, zkoušeno je 150 kombinací hyperparametrů pro každou z variant.

## Import knihoven a základní nastavení

In [1]:
from transformers import BasicTokenizer, Trainer
from datasets import concatenate_datasets, load_from_disk
import kagglehub
import optuna
import torch
import math
import base

Resetování náhodného seedu pro replikovatelnost výsledků.

In [ ]:
base.reset_seed()

Ověření dostupnosti GPU.

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Načtení embeddingů.

Načtení datasetu a jeho základní předzpracování (tokenizace, vytvoření slovníků všech tokenů, vytvoření indexu pro GloVe embeddingy).

In [4]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [ ]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "sst2"

In [6]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits", f"~/data/{DATASET}/test-logits", f"~/data/{DATASET}/train-logits-augmented"]])
tokenizer = BasicTokenizer(do_lower_case=True)

Tokenizace.

In [7]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

Získání všech unikátních tokenů v datasetu.

In [8]:
vocab = base.get_vocab(all_data_tokens)

Přiřazení indexu jednotlivým tokenům.

In [ ]:
word_index = dict(zip(vocab, range(len(vocab))))

Získání indexů z GloVe embeddingů.

In [10]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


Definice velikosti slovníku a velikosti embedding dimenze. 

In [ ]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

14621


Vytvoření vazby mezi tokeny (jejich indexy) a embeddingy. Část tokenů nebyla nalezena, což ovšem nepředstavuje problém.

In [ ]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 14305 words (316) misses


Přiřazení indexu tokenům v každé části datasetu.

In [13]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

Zarovnání délky všech záznamů.

In [14]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))

Přidání ID tokenů do každé části datasetu.

In [15]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

Základní konfigurace tréninku během prohledávání. Optuna nepracuje s epochami, ale s kroky. Níže je prováděn přepočet. 

Minimální délka tréninku jsou tři epochy, maximální 10 epoch. Maximální počet kroků pro warm up je nastaven na 10 % první epochy.

In [ ]:
num_epochs = 10
batch_size = 128

In [ ]:
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*3
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

## Prohledávání s normálním tréninkem nad původním datasetem
Definice hledaných hyperparametrů a jejich rozmezí.

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Získání modelu s definovanou odemčenou embedding vrstvou. 

In [ ]:
def get_BiLSTM():
    return base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=2, freeze_embed=False)

In [ ]:
base.reset_seed()

Konfigurace jednotlivých tréninků.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd_hp-search", epochs=num_epochs, batch_size=batch_size)

Konfigurace trenéra pro jednotlivé tréninky. 

In [23]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM()
)
  

Nastavení prohledávání.

In [ ]:
best_trial_normal = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-embedd",
    n_trials=150
)

[I 2025-03-24 18:30:08,634] A new study created in memory with name: Base-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 32}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.384500,0.417013,0.811927,0.818788,0.813253,0.811306
2,0.242900,0.441692,0.822248,0.828598,0.820946,0.820923
3,0.184600,0.464164,0.827982,0.832648,0.826871,0.826991
4,0.149100,0.550418,0.830275,0.837502,0.828913,0.828906
5,0.123400,0.527008,0.829128,0.829148,0.828966,0.829029
6,0.103600,0.579586,0.834862,0.835999,0.834302,0.834514


[I 2025-03-24 18:31:19,812] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 6}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.348100,0.441757,0.815367,0.822973,0.816757,0.814682
2,0.211600,0.458589,0.827982,0.833026,0.826829,0.826929
3,0.152800,0.460299,0.833716,0.834940,0.833133,0.833347
4,0.117200,0.542018,0.834862,0.837107,0.834091,0.834318
5,0.091900,0.599790,0.833716,0.835352,0.833049,0.833272
6,0.074000,0.649204,0.846330,0.847173,0.845858,0.846068
7,0.058800,0.689745,0.840596,0.840536,0.840606,0.840561
8,0.047700,0.725697,0.833716,0.833795,0.833891,0.833710
9,0.038900,0.790143,0.827982,0.829095,0.828513,0.827949
10,0.032000,0.852410,0.833716,0.833654,0.833723,0.833679


[I 2025-03-24 21:59:11,833] Trial 148 finished with value: 0.8336786303874562 and parameters: {'learning_rate': 0.0004708293252621487, 'weight_decay': 0.005, 'warmup_steps': 0}. Best is trial 71 with value: 0.8462906544170652.


Trial 149 with params: {'learning_rate': 0.0008753621830154515, 'weight_decay': 0.003, 'warmup_steps': 0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.320600,0.524938,0.802752,0.821784,0.804959,0.800532
2,0.180700,0.423844,0.837156,0.838126,0.836638,0.836846
3,0.121400,0.413718,0.849771,0.850261,0.849404,0.849581
4,0.086200,0.536922,0.848624,0.850037,0.848026,0.848272
5,0.062200,0.631359,0.839450,0.842292,0.838596,0.838831
6,0.045400,0.734328,0.831422,0.835605,0.830376,0.830537


[I 2025-03-24 22:00:25,901] Trial 149 pruned. 


In [25]:
print(best_trial_normal)

BestRun(run_id='71', objective=0.8462906544170652, hyperparameters={'learning_rate': 0.0006342434465905245, 'weight_decay': 0.005, 'warmup_steps': 1}, run_summary=None)


In [26]:
base.reset_seed()

## Prohledávání s destilací nad původním datasetem
Konfigurace jednotlivých tréninků.

In [27]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.


In [28]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [29]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [30]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

Nastavení prohledávání.

In [31]:
best_trial_distill = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-embedd",
    n_trials=150
)

[I 2025-03-25 09:28:19,571] A new study created in memory with name: Distill-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 32, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.719100,1.487449,0.807339,0.811110,0.808327,0.807046
2,0.927300,1.317457,0.838303,0.839193,0.837806,0.838011
3,0.661900,1.419669,0.832569,0.837702,0.831418,0.831544


[I 2025-03-25 09:28:55,161] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 38, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.069500,1.224997,0.832569,0.833692,0.833102,0.832537
2,0.378100,1.180843,0.853211,0.853649,0.852867,0.853037
3,0.200300,1.176175,0.854358,0.854999,0.853951,0.854149
4,0.125200,1.229715,0.845183,0.845794,0.844774,0.844961
5,0.083100,1.333551,0.839450,0.839394,0.839480,0.839419
6,0.062500,1.251139,0.848624,0.848573,0.848573,0.848573


[I 2025-03-25 12:49:23,366] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0002803349651617039, 'weight_decay': 0.0, 'warmup_steps': 15, 'lambda_param': 1.0, 'temperature': 4.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.705600,1.489692,0.808486,0.810470,0.809201,0.808375
2,0.937200,1.332799,0.837156,0.837552,0.836806,0.836963
3,0.668500,1.433420,0.829128,0.833631,0.828039,0.828174
4,0.511200,1.465675,0.824541,0.827356,0.823661,0.823840
5,0.403400,1.478586,0.832569,0.832618,0.832723,0.832561
6,0.333400,1.564063,0.838303,0.839554,0.837722,0.837944


[I 2025-03-25 12:50:30,210] Trial 149 pruned. 


In [32]:
print(best_trial_distill)

BestRun(run_id='103', objective=0.8691827872088432, hyperparameters={'learning_rate': 0.0008555235299408995, 'weight_decay': 0.01, 'warmup_steps': 9, 'lambda_param': 0.1, 'temperature': 5.5}, run_summary=None)


Přepočet kroků s ohledem na změnu velikosti datasetu.

In [40]:
data_length = len(all_train_data)
min_r = math.ceil(data_length/batch_size)*3
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [41]:
base.reset_seed()

## Prohledávání s normálním tréninkem nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [42]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd-aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd-aug_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [43]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [44]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [45]:
trainer = Trainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

Nastavení prohledávání.

In [46]:
best_trial_normal_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-aug-embedd",
    n_trials=150
)

[I 2025-03-25 12:50:30,502] A new study created in memory with name: Base-aug-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 169}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.242600,0.497180,0.832569,0.837702,0.831418,0.831544
2,0.110900,0.513967,0.847477,0.847512,0.847320,0.847389
3,0.075200,0.661170,0.848624,0.848564,0.848615,0.848585


[I 2025-03-25 12:51:56,792] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 36}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.219700,0.468791,0.837156,0.838126,0.836638,0.836846
2,0.098200,0.536981,0.837156,0.838126,0.836638,0.836846
3,0.065000,0.672091,0.844037,0.843984,0.843984,0.843984
4,0.046000,0.780488,0.833716,0.833949,0.833975,0.833715
5,0.033300,0.878917,0.833716,0.835139,0.833091,0.833310
6,0.024400,1.023829,0.827982,0.830983,0.827082,0.827269


[I 2025-03-25 21:30:13,840] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.00020709448845232986, 'weight_decay': 0.009000000000000001, 'warmup_steps': 116}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.258100,0.467387,0.825688,0.828104,0.824872,0.825066
2,0.127600,0.502365,0.841743,0.841856,0.841942,0.841740
3,0.089800,0.655639,0.850917,0.851204,0.851204,0.850917
4,0.068800,0.713962,0.842890,0.844332,0.843490,0.842843
5,0.054100,0.801374,0.834862,0.836856,0.834133,0.834360
6,0.043000,0.814185,0.838303,0.839973,0.837638,0.837871
7,0.035100,0.883581,0.842890,0.843054,0.842648,0.842761
8,0.028300,1.057064,0.830275,0.831576,0.829671,0.829881
9,0.023200,1.172056,0.834862,0.835055,0.834596,0.834715
10,0.019300,1.249564,0.832569,0.832617,0.832386,0.832462


[I 2025-03-25 21:35:07,218] Trial 149 finished with value: 0.8324621660744835 and parameters: {'learning_rate': 0.00020709448845232986, 'weight_decay': 0.009000000000000001, 'warmup_steps': 116}. Best is trial 73 with value: 0.8416756571849591.


In [ ]:
print(best_trial_normal_aug)

BestRun(run_id='73', objective=0.8416756571849591, hyperparameters={'learning_rate': 0.00017049928990776612, 'weight_decay': 0.01, 'warmup_steps': 30}, run_summary=None)


In [ ]:
base.reset_seed()

## Prohledávání s destilací nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd-aug_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd-aug_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [ ]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [ ]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [ ]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

Nastavení prohledávání.

In [ ]:
best_trial_distill_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-aug-embedd",
    n_trials=150
)

[I 2025-03-25 21:35:07,737] A new study created in memory with name: Distill-aug-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 169, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.841700,1.145580,0.850917,0.853593,0.850109,0.850385
2,0.298900,1.124657,0.868119,0.868532,0.867801,0.867973
3,0.203200,1.145954,0.869266,0.869293,0.869138,0.869197
4,0.157700,1.162383,0.866972,0.867804,0.867433,0.866961
5,0.128000,1.151717,0.860092,0.860071,0.860003,0.860032
6,0.106500,1.056618,0.875000,0.874970,0.874937,0.874952


[I 2025-03-25 21:38:08,945] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 200, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.731300,1.123153,0.858945,0.859756,0.858498,0.858717
2,0.257700,1.049398,0.865826,0.865877,0.865675,0.865748
3,0.174400,1.126699,0.862385,0.862846,0.862044,0.862222


[I 2025-03-26 07:26:36,548] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0001627020201335242, 'weight_decay': 0.007, 'warmup_steps': 60, 'lambda_param': 0.6000000000000001, 'temperature': 4.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.971700,1.199179,0.840596,0.842283,0.839932,0.840171
2,0.407200,1.138544,0.863532,0.863612,0.863718,0.863528
3,0.278300,1.218563,0.866972,0.866955,0.866886,0.866916
4,0.219100,1.346574,0.856651,0.859119,0.857424,0.856552
5,0.183300,1.189411,0.870413,0.870536,0.870222,0.870323
6,0.157100,1.155966,0.872706,0.872832,0.872516,0.872618
7,0.138900,1.239231,0.868119,0.868806,0.867717,0.867930
8,0.124900,1.254968,0.872706,0.872832,0.872516,0.872618
9,0.114000,1.269183,0.864679,0.864834,0.864465,0.864576
10,0.106500,1.262300,0.865826,0.866024,0.865591,0.865715


[I 2025-03-26 07:32:01,083] Trial 149 finished with value: 0.8657153123556285 and parameters: {'learning_rate': 0.0001627020201335242, 'weight_decay': 0.007, 'warmup_steps': 60, 'lambda_param': 0.6000000000000001, 'temperature': 4.5}. Best is trial 31 with value: 0.8749524730461324.


In [ ]:
print(best_trial_distill_aug)

BestRun(run_id='31', objective=0.8749524730461324, hyperparameters={'learning_rate': 0.0002255603737182001, 'weight_decay': 0.005, 'warmup_steps': 63, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}, run_summary=None)
